# Implement CDC: Change Data Capture
## Use-case: Synchronize your SQL Database with your Lakehouse

Delta Lake is an <a href="https://delta.io/" target="_blank">open-source</a> storage layer with Transactional capabilities and increased Performances. 

Delta lake is designed to support CDC workload by providing support for UPDATE / DELETE and MERGE operation.

In addition, Delta table can support CDC to capture internal changes and propagate the changes downstream.

## CDC flow

Here is the flow we'll implement, consuming CDC data from an external database. 

Note that the incoming could be any format, including message queue such as Kafka.


<img width="1000px" src="https://github.com/databricks-demos/dbdemos-resources/raw/main/images/product/Delta-Lake-CDC-CDF/cdc-flow-0.png" alt='Make all your data ready for BI and ML'/>

In [0]:
%run ./_resources/00-setup $reset_all_data=false

## Bronze: Incremental data loading using Auto Loader.

<img src="https://github.com/databricks-demos/dbdemos-resources/raw/main/images/product/Delta-Lake-CDC-CDF/cdc-flow-1.png" alt='Make all your data ready for BI and ML' style='float: right' width='600'/>

Working with external system can be challenging due to schema update. The external database can have schema update, adding or modifying columns, and our system must be robust against these changes.

Databricks Autoloader (`cloudFiles`) handles schema inference and evolution out of the box.

In [0]:
# Have a look at raw CSV files:
cdc_raw_data = (spark.read
                      .option("header", "true")
                      .csv(raw_data_location + "/user_csv"))

display(cdc_raw_data)

In [0]:
# Our CDC is sending 3 types of operation: APPEND, DELETE and UPDATE.
display(cdc_raw_data.select("operation").distinct())

In [0]:
# Read CSV files and write them to delta bronze tables:
bronzeDF = (spark.readStream
            .format("cloudFiles")
            .option("cloudFiles.format", "csv")
            .option("cloudFiles.maxFilesPerTrigger", "1") # Simulate streaming, remove in production.
            .option("cloudFiles.inferColumnTypes", "true")
            .option("cloudFiles.schemaLocation", raw_data_location + "/stream/schema_cdc_raw")
            .option("cloudFiles.schemaHints", "id BIGINT, operation_date TIMESTAMP")
            .load(raw_data_location + "/user_csv")
                .withColumn("file_name", F.col("_metadata.file_path")))

(bronzeDF.writeStream
            .option("checkpointLocation", raw_data_location + "/stream/checkpoint_cdc_raw")
            .trigger(processingTime = "10 seconds")
            .table("clients_cdc"))

time.sleep(20)

In [0]:
%sql
DESCRIBE EXTENDED clients_cdc;

In [0]:
%sql
-- Let's make sure our table has the proper compaction settings to support streaming:
ALTER TABLE clients_cdc
  SET TBLPROPERTIES (
    delta.autoOptimize.optimizeWrite = true,
    delta.autoOptimize.autoCompact = true);

SELECT * FROM clients_cdc
ORDER BY id ASC;

## Silver: Materialize the table.

<img src="https://github.com/databricks-demos/dbdemos-resources/raw/main/images/product/Delta-Lake-CDC-CDF/cdc-flow-2.png" alt='Make all your data ready for BI and ML' style='float: right' width='600'/>

The silver `retail_client_silver` table will contains the most up to date view. It'll be a replicate of the original MYSQL table.

Because we'll propagate the `MERGE` operations downstream to the `GOLD` layer, we need to enable Delta Lake CDF: `delta.enableChangeDataFeed = true`

In [0]:
%sql
-- Create our client silver table using standard SQL command:
CREATE TABLE IF NOT EXISTS retail_client_silver (
    id BIGINT NOT NULL,
    name STRING,
    address STRING,
    email STRING,
    operation STRING
  )
  TBLPROPERTIES (
    delta.enableChangeDataFeed = true,
    delta.autoOptimize.optimizeWrite = true,
    delta.autoOptimize.autoCompact = true
  );

In [0]:
# Run the MERGE statement to upsert CDC information into our silver table:
def merge_stream(df, i):
    df.createOrReplaceTempView("clients_cdc_microbatch")

    # First, we need to dedup the incoming data based on ID (we can have multiple update of the same row in our incoming data).
    #Then we run the merge (upsert or delete).
    df.sparkSession.sql("""
                            MERGE INTO retail_client_silver AS t
                                USING (SELECT id, name, address, email, operation
                                       FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY id 
                                                                          ORDER BY operation_date DESC) AS rnk
                                             FROM clients_cdc_microbatch)
                                       WHERE rnk = 1) AS s
                                ON t.id = s.id
                                WHEN MATCHED AND s.operation = 'DELETE' THEN DELETE
                                WHEN MATCHED AND s.operation != 'DELETE' THEN UPDATE SET *
                                WHEN NOT MATCHED AND s.operation != 'DELETE' THEN INSERT *
                        """)

(spark.readStream
        .table("clients_cdc")
      .writeStream
        .foreachBatch(merge_stream)
        .option("checkpointLocation", raw_data_location + "/stream/checkpoint_clients_cdc")
        .trigger(processingTime = "10 seconds")
        .start())

time.sleep(20)

In [0]:
%sql
SELECT * FROM retail_client_silver
ORDER BY id ASC;

### Testing the first CDC layer:

In [0]:
%sql
INSERT INTO clients_cdc (id, name, address, email, operation_date, operation, _rescued_data, file_name) 
VALUES (1000, "Quentin", "Paris 75020", "quentin.ambard@databricks.com", now(), "UPDATE", null, null),
       (2000, null, null, null, now(), "DELETE", null, null);

SELECT * FROM clients_cdc WHERE id IN (1000, 2000);

In [0]:
%sql
SELECT * FROM retail_client_silver WHERE id IN (1000, 2000);

## Gold: Capture and Propagate Silver modifications downstream.

<img src="https://github.com/databricks-demos/dbdemos-resources/raw/main/images/product/Delta-Lake-CDC-CDF/cdc-flow-3.png" alt='Make all your data ready for BI and ML' style='float: right' width='600'/>

We need to add a final Gold layer based on the data from the Silver table. If a row is DELETED or UPDATED in the SILVER layer, we want to apply the same modification in the GOLD layer.

To do so, we need to capture all the tables changes from the SILVER layer and incrementally replicate the changes to the GOLD layer.

This is very simple using Delta Lake CDF from our SILVER table!

Delta Lake CDF provides the `table_changes('< table_name >', < delta_version >)` that you can use to select all the tables modifications from a specific Delta version to another one:

In [0]:
%sql
-- Delta Lake CDF works using table_changes function:
SELECT * FROM table_changes('retail_client_silver', 1)
ORDER BY id;

#### Delta CDF table_changes output
Table Changes provides back 4 cdc types in the "_change_type" column:

| CDC Type             | Description                                                               |
|----------------------|---------------------------------------------------------------------------|
| **update_preimage**  | Content of the row before an update                                       |
| **update_postimage** | Content of the row after the update (what you want to capture downstream) |
| **delete**           | Content of a row that has been deleted                                    |
| **insert**           | Content of a new row that has been inserted                               |

Therefore, 1 update will result in 2 rows in the cdc stream (one row with the previous values, one with the new values)

In [0]:
from delta.tables import DeltaTable

latest_version = str(DeltaTable.forName(spark, "retail_client_silver").history(1).head()["version"])

print(f"our Delta table last version is {latest_version}, let's select the last changes to see our DELETE and UPDATE operations (last 2 versions):")

changes = (spark.read
                    .format("delta")
                    .option("readChangeData", "true")
                    .option("startingVersion", int(latest_version) - 1)
                    .table("retail_client_silver"))

display(changes)

### Synchronizing our downstream GOLD table based from the Silver changes

Let's now say that we want to perform another table enhancement and propagate these changes downstream.

To keep this example simple, we'll just add a column name `gold_data` with random data, but in real world this could be an aggregation, a join with another datasource, an ML model etc.

In [0]:
%sql
CREATE TABLE IF NOT EXISTS retail_client_gold (
  id BIGINT NOT NULL,
  name STRING,
  address STRING,
  email STRING,
  gold_data STRING
);

In [0]:
from pyspark.sql.window import Window

# Function to upsert `microBatchOutputDF` into Delta table using MERGE:
def upsertToDelta(data, batchId):
    # Need to deduplicate based on the id and take the most recent update:
    windowSpec = Window.partitionBy("id").orderBy(F.col("_commit_version").desc())
    data_deduplicated = (data.withColumn("rnk", F.dense_rank().over(windowSpec))
                             .where("rnk = 1 and _change_type != 'update_preimage'")
                             .drop("_commit_version", "rnk"))
    
    # Add some data cleaning for the gold layer to remove quotes from the address:
    data_deduplicated = data_deduplicated.withColumn("address", F.regexp_replace(F.col("address"), "\"", ""))

    # Run the merge in the gold table directly:
    (DeltaTable.forName(spark, "retail_client_gold").alias("t")
        .merge(data_deduplicated.alias("s"), "s.id = t.id")
        .whenMatchedDelete("s._change_type = 'delete'")
        .whenMatchedUpdateAll("s._change_type != 'delete'")
        .whenNotMatchedInsertAll()
        .execute())

(spark.readStream
        .option("readChangeData", "true")
        .option("startingVersion", 1)
        .table("retail_client_silver")
        .withColumn("gold_data", F.lit("CDF!"))
      .writeStream
        .foreachBatch(upsertToDelta)
        .option("checkpointLocation", raw_data_location + "/stream/checkpoint_clients_gold")
        .trigger(processingTime = "10 seconds")
        .start())

In [0]:
%sql
SELECT * FROM retail_client_gold;

In [0]:
DBDemos.stop_all_streams()